In [107]:
from gekko import GEKKO
import numpy as np

#### 問題設定 ####
farmer_num = 5 # 農家の数
ware_house_num = 2 # 倉庫の数
customer_num = 4 # 客の数

supry = np.array([10,5,2,7,4]) # 農家の方の供給
demand = np.array([7, 8, 3, 5]) # 店の方の需要
a = np.array([2, 3]) #倉庫で腐れ落ちる数
c_bar_np = np.array([[2.,4.], [2.,4.],[2.,4.],[2.,4.],[2,4 ]]) # 輸送コスト 農家から倉庫
c_np = np.array([[2.,2.,2.,2.],[2.,2.,2.,2.]]) # 輸送コスト 倉庫から消費者

##################
#Initialize Model
m = GEKKO()

In [108]:
#後からいじる可能性があるパラメタはm.Param?
supry_pa = m.Array(m.Param,(supry.shape))
for k in range(len(supry)):
    supry_pa[k].value = supry[k]

demand_pa = m.Array(m.Param,(demand.shape))
for i in range(len(demand)):
    demand_pa[i].value = demand[i]

In [109]:
#変数の初期化
gum_bar = m.Array(m.Var,(farmer_num, ware_house_num,))
for k in range(farmer_num):
  for j in range(ware_house_num):
    gum_bar[k,j].value = 2.0
    gum_bar[k,j].lower = 0.0
    #s[i,j].upper = 100.0

gum = m.Array(m.Var,(ware_house_num, customer_num))
for j in range(ware_house_num):
  for i in range(customer_num):
    gum[j,i].value = 2.0
    gum[j,i].lower = 0.0

In [110]:
#コストのアレイ作っとく
c_bar = m.Array(m.Const,(c_bar_np.shape))
for k in range(c_bar_np.shape[0]):
  for j in range(c_bar_np.shape[1]):
    c_bar[k, j].value = c_bar_np[k, j]

c = m.Array(m.Const,(c_np.shape))
for j in range(c_np.shape[0]):
  for i in range(c_np.shape[1]):
    c[j, i].value = c_np[j, i]

In [111]:
# 最適化関数
m.Obj(m.sum([m.sum([gum_bar[k, j] * c_bar[k, j] for k in range(farmer_num)]) for j in range(ware_house_num)]) + \
     m.sum([m.sum([gum[jj, i] * c[jj, i] for jj in range(ware_house_num)]) for i in range(customer_num)]))

In [112]:
# 制約式
for k in range(farmer_num):
    m.Equation(m.sum([gum_bar[k,j] for j in range(ware_house_num)]) == supry_pa[k])
for j in range(ware_house_num):
    m.Equation(m.sum([gum[j,i] for i in range(customer_num)]) == m.sum([gum_bar[k, j] for k in range(farmer_num)]) - a[j])
for i in range(customer_num):
    m.Equation(m.sum([gum[j, i] for j in range(ware_house_num)]) == demand_pa[i])

In [113]:
# 追加の制約 (倉庫の上限)
w_capa = np.array([15, 15])
w_capa_pa = m.Array(m.Param,(w_capa.shape))
for j in range(ware_house_num):
    w_capa_pa[j].value = w_capa[j]
for j in range(ware_house_num):
    m.Equation(m.sum([gum_bar[k, j] for k in range(farmer_num)]) <= w_capa_pa[j])

In [114]:
m.solve()

apm 223.29.218.155_gk_model13 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           23
   Constants    :           18
   Variables    :           72
   Intermediates:            0
   Connections  :           93
   Equations    :           32
   Residuals    :           32
 
 Number of state variables:             61
 Number of total equations: -           54
 Number of slack variables: -            2
 ---------------------------------------
 Degrees of freedom       :              5
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program c

In [115]:
print(gum_bar)
print("================")
print(gum)

[[[5.3866411135] [4.6133588865]]
 [[2.6690497949] [2.3309502051]]
 [[1.0630504884] [0.93694951163]]
 [[3.7493971428] [3.2506028572]]
 [[2.1318614553] [1.8681385447]]]
[[[3.9367935457] [4.4925324375] [1.7377048259] [2.8329691858]]
 [[3.0632064543] [3.5074675625] [1.2622951741] [2.1670308142]]]
